In [38]:
%%time
import pandas as pd
import numpy as np
from datetime import datetime

saldo_clientes = pd.read_csv('SaldoClientes.csv', sep=';', encoding='latin1')
saldo_clientes['Ref Fatura'] = pd.to_datetime(saldo_clientes['Ref Fatura'], format="%Y-%m-%d")
saldo_clientes['Emissão'] = pd.to_datetime(saldo_clientes['Emissão'], format="%Y-%m-%d")
saldo_clientes['Vencimento'] = pd.to_datetime(saldo_clientes['Vencimento'], format="%Y-%m-%d")
data_fim_mes = datetime(2022, 2, 28)
saldo_clientes['Teste'] = (saldo_clientes['Vencimento'] - data_fim_mes) / np.timedelta64(1, 'D')
saldo_clientes['Situação'] = ['Vencida' if x < 0 else 'A vencer' for x in saldo_clientes['Teste']]

vencidas = [0, 30, 60, 90, 120, 150, 180, 360, 999999]
labels_vencidas = ['1 a 30 dias', 
                   '31 a 60 dias', 
                   '61 a 90 dias', 
                   '91 a 120 dias', 
                   '121 a 150 dias', 
                   '151 a 180 dias', 
                   '181 a 360 dias', 
                   'Acima de 361 dias']
a_vencer = [0, 7, 15, 23, 30, 60, 999999]
labels_a_vencer = ['0 a 7 dias',
                   '8 a 15 dias',
                   '16 a 23 dias',
                   '24 a 30 dias',
                   '31 a 60 dias',
                   'mais de 61 dias']

lista_vencidas = saldo_clientes.query('Situação == "Vencida"')[['Teste', 'Situação']]
lista_a_vencer = saldo_clientes.query('Situação == "A vencer"')[['Teste', 'Situação']]
lista_vencidas['Faixa'] = pd.cut(abs(lista_vencidas['Teste']), vencidas, labels = labels_vencidas, include_lowest=True)
lista_a_vencer['Faixa'] = pd.cut(abs(lista_a_vencer['Teste']), a_vencer, labels = labels_a_vencer, include_lowest=True)
saldo_clientes['Faixa'] = pd.concat([lista_a_vencer, lista_vencidas]).sort_index()['Faixa']
saldo_clientes.rename(columns= {' Valor ': 'Valor'}, inplace=True)
saldo_clientes['Valor'] = saldo_clientes['Valor'].str.replace('.','')
saldo_clientes['Valor'] = saldo_clientes['Valor'].str.replace(',','.')
saldo_clientes['Valor'] = saldo_clientes['Valor'].astype('float')
saldo_clientes['REF2'] = saldo_clientes['Vencimento'] - pd.DateOffset(months=1)
saldo_clientes['REF2'] = saldo_clientes['REF2'].apply(lambda dt: dt.replace(day=1))

ref_na = saldo_clientes[saldo_clientes['Ref Fatura'].isna()][['Ref Fatura', 'Emissão']]
ref_na['REF'] = saldo_clientes['Emissão'].apply(lambda dt: dt.replace(day=1))
ref_notna = saldo_clientes[saldo_clientes['Ref Fatura'].notna()][['Ref Fatura', 'Emissão']]
ref_notna['REF'] = saldo_clientes['Ref Fatura']
saldo_clientes['REF']= pd.concat([ref_na, ref_notna]).sort_index()['REF']

# saldo_clientes['Matrícula-Dígito'] = str(saldo_clientes['Matrícula']) + '-' + str(saldo_clientes['Dígito'])

saldo_clientes['Dívida total'] = saldo_clientes.groupby('Matrícula-Dígito')['Valor'].transform(np.sum)

faixa_vencidas = ['1 a 30 dias',
                  '31 a 60 dias',
                  '61 a 90 dias',
                  '91 a 120 dias',
                  '121 a 150 dias',
                  '151 a 180 dias',
                  '181 a 360 dias',
                  'Acima de 361 dias']
faixa_a_vencer = ['0 a 7 dias',
                  '8 a 15 dias',
                  '16 a 23 dias',
                  '24 a 30 dias',
                  '31 a 60 dias', 
                  'mais de 61 dias']

colunas_aging = ['Comercial',
                'Industrial',
                'Público',
                'Residencial',
                'Cobrança Diversa',
                 'Serviço a Faturar']

aging_vencida_resumido = pd.DataFrame(saldo_clientes.query('Situação == "Vencida"').groupby('Faixa')['Valor'].sum(), 
                                      index=faixa_vencidas)
aging_vencida_a_vencer = pd.DataFrame(saldo_clientes.query('Situação == "A vencer"').groupby('Faixa')['Valor'].sum(), 
                                      index=faixa_a_vencer)

aging_a_vencer = pd.DataFrame(pd.crosstab(saldo_clientes.query('Situação == "A vencer"')['Faixa'],
                                          saldo_clientes.query('Situação == "A vencer"')['Categoria2'],
                                          values=saldo_clientes.query('Situação == "A vencer"')['Valor'],
                                          aggfunc='sum'),
                              index=faixa_a_vencer,
                              columns=colunas_aging)

aging_a_vencer = aging_a_vencer.fillna(0)

aging_vencido = pd.DataFrame(pd.crosstab(saldo_clientes.query('Situação == "Vencida"')['Faixa'],
                                         saldo_clientes.query('Situação == "Vencida"')['Categoria2'],
                                         values=saldo_clientes.query('Situação == "Vencida"')['Valor'],
                                         aggfunc='sum'),
                             index=faixa_vencidas,
                             columns=colunas_aging)
aging_vencido = aging_vencido.fillna(0)

<timed exec>:36: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


Wall time: 12.4 s


In [39]:
round(aging_vencida_resumido.sum() - aging_vencido.sum().sum() , 2)

Valor   -0.0
dtype: float64

In [40]:
round(aging_vencida_a_vencer.sum() - aging_a_vencer.sum().sum(), 2)

Valor   -0.0
dtype: float64

In [41]:
aging_vencida_resumido

,Valor
1 a 30 dias,5211378.16
31 a 60 dias,1425077.68
61 a 90 dias,561915.01
91 a 120 dias,387075.64
121 a 150 dias,358677.72
151 a 180 dias,412984.71
181 a 360 dias,2227720.35
Acima de 361 dias,12472383.24


In [42]:
aging_vencida_a_vencer

,Valor
0 a 7 dias,8316996.60
8 a 15 dias,7074171.01
16 a 23 dias,5662944.63
24 a 30 dias,200732.65
31 a 60 dias,415916.00
mais de 61 dias,3769572.29


In [43]:
aging_a_vencer

,Comercial,Industrial,Público,Residencial,Cobrança Diversa,Serviço a Faturar
0 a 7 dias,1667770.47,645670.22,11704.37,5987308.32,0.0,4543.22
8 a 15 dias,554209.95,749211.92,6199.25,5750600.68,0.0,13949.21
16 a 23 dias,544859.19,126784.57,929011.87,4056999.84,0.0,5289.16
24 a 30 dias,17035.39,2598.33,53471.02,120833.50,0.0,6794.41
31 a 60 dias,51900.93,6221.28,248.09,345356.33,0.0,12189.37
mais de 61 dias,722130.02,33202.13,5263.03,2947113.36,0.0,61863.75


In [44]:
aging_vencido

,Comercial,Industrial,Público,Residencial,Cobrança Diversa,Serviço a Faturar
1 a 30 dias,580945.30,116251.13,5304.29,4485115.94,17411.52,6349.98
31 a 60 dias,184889.79,6849.03,767.30,1231374.48,0.00,1197.08
61 a 90 dias,82482.72,4373.25,276.10,474782.94,0.00,0.00
91 a 120 dias,65607.17,10694.52,254.33,310519.62,0.00,0.00
121 a 150 dias,59038.87,3150.08,470.29,295807.11,211.37,0.00
151 a 180 dias,71403.66,3881.79,350.54,335837.74,416.41,1094.57
181 a 360 dias,365961.94,33624.50,1944.11,1825212.05,977.75,0.00
Acima de 361 dias,2615931.63,285816.52,970253.08,8553069.43,36415.86,10896.72


In [45]:
saldo_clientes.to_excel('SaldoClientes_atualizado.xlsx')